In [1]:
# Imports
import argparse
import torch
import torchvision
from tqdm import tqdm
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import torchvision.transforms as transforms
from matplotlib import image
from matplotlib import pyplot
import pandas as pd
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import RandomSampler
from torchvision.datasets import ImageFolder


In [2]:
DIR_TRAIN = "./train/"
DIR_VALID = "./valid/"
DIR_TEST = "./test/"
classes = os.listdir(DIR_TRAIN)
print("Total Classes: ",len(classes))

#Counting total train, valid & test images

train_count = 0
valid_count = 0
test_count = 0
for _class in classes:
    train_count += len(os.listdir(DIR_TRAIN + _class))
    valid_count += len(os.listdir(DIR_VALID + _class))
    test_count += len(os.listdir(DIR_TEST + _class))

print("Total train images: ",train_count)
print("Total valid images: ",valid_count)
print("Total test images: ",test_count)

Total Classes:  200
Total train images:  29435
Total valid images:  1000
Total test images:  1000


In [3]:
train_imgs = []
valid_imgs = []
test_imgs = []

for _class in classes:
    
    for img in os.listdir(DIR_TRAIN + _class):
        train_imgs.append(DIR_TRAIN + _class + "/" + img)
    
    for img in os.listdir(DIR_VALID + _class):
        valid_imgs.append(DIR_VALID + _class + "/" + img)
        
    for img in os.listdir(DIR_TEST + _class):
        test_imgs.append(DIR_TEST + _class + "/" + img)

class_to_int = {classes[i] : i for i in range(len(classes))}

In [4]:
train_dataset = ImageFolder(root = DIR_TRAIN, transform = T.ToTensor())
valid_dataset = ImageFolder(root = DIR_VALID, transform = T.ToTensor())
test_dataset = ImageFolder(root = DIR_TEST, transform = T.ToTensor())

In [5]:
model_res = models.resnet18(pretrained=True)
# turn training false for all layers, other than fc layer
for param in model_res.parameters():
    param.requires_grad = False
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        # TODO
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*224*224, 2048),
            nn.ReLU(),
            nn.Dropout(.35),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(.35),
            nn.Linear(512, 200),
        )

    def forward(self, x):
        # TODO
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
def set_random(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
def get_args():
    parser = argparse.ArgumentParser(description='Args for training networks')
    parser.add_argument('-seed', type=int, default=1, help='random seed')
    parser.add_argument('-num_epochs', type=int, default=20, help='num epochs')
    parser.add_argument('-batch', type=int, default=16, help='batch size')
    parser.add_argument('-lr', type=float, default=0.01, help='learning rate')
    parser.add_argument('-drop', type=float, default=0.3, help='drop rate')
    args, _ = parser.parse_known_args()
    return args
def load_data(batch_size=16):
    trainset = train_dataset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=64,
        shuffle=True,
        #sampler=train_random_sampler,
        num_workers=10,pin_memory=True)
    testset = test_dataset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=32,
        shuffle=False,
        #sampler=test_random_sampler,
        num_workers=10,pin_memory=True)
    validset = valid_dataset
    validloader = torch.utils.data.DataLoader(
        validset, batch_size=32, 
        shuffle=False,
        #sampler=valid_random_sampler,
        num_workers=10, pin_memory=True)
    return trainloader, testloader, validloader

In [7]:
def train(net, trainloader, num_epochs=2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(num_epochs):
        net.train()
        running_loss, total,  correct = 0.0, 0, 0
        for images, labels in tqdm(
                trainloader, desc='Epoch '+str(epoch), unit='b'):
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

        loss = running_loss / len(trainloader)
        accuracy = 100 * correct / total
        print('Epoch %d training loss: %.3f training accuracy: %.3f%%' % (
            epoch, loss, accuracy))


def test(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for images, labels in tqdm(testloader, desc='Test', unit='b'):
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test set: %.2f%%' % (
        100 * correct / total))

args = get_args()
set_random(args.seed)
#trainloader, testloader = load_data(args.batch)
trainloader, testloader, validloader = load_data(args.batch)
for X, y in testloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
#net = NNet()
model_res.fc=nn.Linear(512,200)
### Get device

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()

model_res.to(device)
train(model_res, trainloader, 10)
test(model_res, testloader)

Shape of X [N, C, H, W]:  torch.Size([32, 3, 224, 224])
Shape of y:  torch.Size([32]) torch.int64


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████| 460/460 [15:01<00:00,  1.96s/b]


Epoch 0 training loss: 4.632 training accuracy: 16.032%


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 460/460 [14:57<00:00,  1.95s/b]


Epoch 1 training loss: 3.345 training accuracy: 51.082%


...

Epoch 8 training loss: 1.008 training accuracy: 86.339%


Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 460/460 [14:32<00:00,  1.90s/b]


Epoch 9 training loss: 0.931 training accuracy: 87.026%


Test: 100%|█████████████████████████████████████████████████████████████████████████████| 32/32 [00:31<00:00,  1.01b/s]

Accuracy of the network on the 10000 test set: 91.20%
